In [262]:
import re
import numpy as np
myfile = open("macmorpho-train.txt", "r")
line = myfile.readline() 

In [263]:
filename = "macmorpho-train.txt"

In [264]:
tags = {}
vocab = {}
j = 0
tags['<S>'] = 0
i = 1 


with open(filename, "r", encoding = "utf-8") as file:
    for line in file:
        strippedLine = line.strip()
        preLine = "<S> " + strippedLine + " <E>"
        for w in preLine.split():
            matches = re.findall(r'_[^_]+$', w)
            
            for match in matches:
                prepW = w.replace(match, "")
                
                if(prepW not in vocab):
                    vocab[prepW] = j
                    j+= 1
                
                if(match not in tags):
                    tags[match] = i
                    i += 1
                previous = match

A = np.zeros((len(tags), len(tags)))
B = np.zeros((len(tags), len(vocab)))


with open(filename, "r", encoding = "utf-8") as file:
    for line in file:
        strippedLine = line.strip()
        preLine = "<S> " + strippedLine + " <E>"       
        previous = "<S>"
        for w in preLine.split():
            matches = re.findall(r'_[^_]+$', w)
            for match in matches:
                prepW = w.replace(match, "")
                previousIndex = tags[previous]
                currentIndex = tags[match]
                wordIndex = vocab[prepW]
                
                A[previousIndex][currentIndex] += 1
                B[currentIndex][wordIndex] += 1
                
                previous = match

In [265]:
B_prob = np.zeros((len(tags), len(vocab)))

for tag in tags:
    appNumberTag = np.sum(B[tags[tag]])
    for word in vocab:
        appNumberWord = B[tags[tag]][vocab[word]]
        if(appNumberTag == 0):
            B_prob[tags[tag]][vocab[word]] = 0
        else:
            B_prob[tags[tag]][vocab[word]] = (appNumberWord)/(appNumberTag) 

In [266]:
A_prob = np.zeros((len(tags), len(tags)))
numberOfStarts = np.sum(A[tags['<S>']])

for tag in tags:
    appNumberAfter = A[tags['<S>']][tags[tag]]
    A_prob[tags['<S>']][tags[tag]] = appNumberAfter/numberOfStarts

for tagBefore in tags:
    if(tagBefore == '<S>'):
        continue
    appNumberTag = np.sum(B[tags[tagBefore]])
    for tagAfter in tags:
        if(tagAfter != '<S>'):
            appNumberAfter = A[tags[tagBefore]][tags[tagAfter]]
            A_prob[tags[tagBefore]][tags[tagAfter]] = appNumberAfter/appNumberTag
            

In [277]:
tagLess = tags.copy()
tagLess.pop("<S>")
sentence = "Estudar linguagem é muito interessante"
sentenceList = sentence.split()
viterbi = np.zeros((len(tags), len(sentenceList)))
backpointer = np.zeros((len(tags), len(sentenceList)))

In [278]:
for tag in tagLess:
    tagIndex = tagLess[tag]
    viterbi[tagIndex][0] = A_prob[tags['<S>']][tagLess[tag]]*B[tags[tag]][vocab[sentenceList[0]]]
    backpointer[tagIndex][0] = 0
    
for t in range(1, len(sentenceList)):
    word = sentenceList[t]
    
    for tag in tagLess:
        viterbi[tagLess[tag]][t] = ((viterbi[:,t-1]*A_prob[:,tags[tag]]).max())*B_prob[tags[tag]][vocab[word]]
        backpointer[tags[tag]][t] = (viterbi[:,t-1]*A_prob[:,tags[tag]]).argmax()
    
bestpointer = viterbi[:,-1].argmax()

stack = []

for  t in reversed(range(0, len(sentenceList))):
    key = list(tags.keys())[list(tags.values()).index(bestpointer)]
    
    stack.append(f'{sentenceList[t]}: {key}')
    
    bestpointer = int(backpointer[bestpointer][t])


In [279]:
while(len(stack) > 0):
    print(stack.pop())

Estudar: _V
linguagem: _N
é: _V
muito: _ADV
interessante: _ADJ
